In [2]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
%load_ext version_information
%version_information numpy, pandas, matplotlib
%version_information
fig_prefix = "../figures/2018-07-22-jw-weaselball-heatmap_"
data_prefix = "../data/2018-07-22-jw-weaselball-heatmap_"


In [3]:
df1 = pd.read_csv('../../data/07-28-2018_22-55-56.csv', index_col=False)
print(df1.shape)
df2 = pd.read_csv('../../data/07-29-2018_17-26-37.csv', index_co
                  
                  
                  
                  l=False)
print(df2.shape)
df3 = pd.read_csv('../../data/07-29-2018_13-13-11.csv', index_col=False)
print(df3.shape)
df4 = pd.read_csv('../../data/07-29-2018_22-44-40.csv', index_col=False)
print(df4.shape)
df5 = pd.read_csv('../../data/07-30-2018_03-47-59.csv', index_col=False)
print(df5.shape)
frames = [df1,df2,df3,df4,df5]
df = pd.concat(frames,ignore_index=True)
df.head()


(19274541, 7)
(6492777, 7)
(6152540, 7)
(17329453, 7)
(12237448, 7)


,Time,ID,X,Y,Yaw,ResetID,checkCorrectness
0,0 202000000,0,0.000000,0.000000,0.000000,4.0,1.0
1,0 203000000,0,-0.012095,0.012400,-0.098138,4.0,1.0
2,0 204000000,0,-0.003114,-0.052982,-0.181063,4.0,1.0
3,0 205000000,0,-0.002668,-0.053384,-0.182874,4.0,1.0
4,0 206000000,0,-0.001664,-0.053407,-0.186672,4.0,1.0


In [4]:
#Sample the data
SAMPLING_RATE = 200 #Keep 1 row for every SAMPLING_RATE
df_sampled = df.iloc[::SAMPLING_RATE,:]
print("Size of new DF is {}".format(df_sampled.shape))
df_sampled.head(10)

Size of new DF is (307434, 7)


,Time,ID,X,Y,Yaw,ResetID,checkCorrectness
0,0 202000000,0,0.000000,0.000000,0.000000,4.0,1.0
200,0 402000000,0,0.145788,-0.097673,-0.801413,4.0,1.0
400,0 602000000,0,0.256540,-0.132600,-1.436840,4.0,1.0
600,0 802000000,0,0.328300,-0.121642,-2.074360,4.0,1.0
800,1 2000000,0,0.386734,-0.131699,-2.530190,4.0,1.0
1000,1 202000000,0,0.388289,-0.126979,-2.664740,4.0,1.0
1200,0 7000000,0,0.385060,0.384344,2.068000,5.0,1.0
1400,0 207000000,0,0.385988,0.383226,2.082380,5.0,1.0
1600,0 407000000,0,0.388295,0.382646,2.097350,5.0,1.0
1800,0 607000000,0,0.387792,0.377979,1.969430,5.0,1.0


In [6]:
#Clean x,y data
df_clean = df_sampled.copy()
#Clean up the data

#Shift X and Y over so that way it could be made easier to use. Currently the world reference is at the center of the "play area"
LENGTH_OF_BOX = 1.127 #This can be obtained from the .sdf file of the weazelball enclosure in gazebo

df_clean['X'] += LENGTH_OF_BOX / 2
df_clean['Y'] += LENGTH_OF_BOX / 2
df_clean.head()

,Time,ID,X,Y,Yaw,ResetID,checkCorrectness
0,0 202000000,0,0.563500,0.563500,0.000000,4.0,1.0
200,0 402000000,0,0.709288,0.465827,-0.801413,4.0,1.0
400,0 602000000,0,0.820040,0.430900,-1.436840,4.0,1.0
600,0 802000000,0,0.891800,0.441858,-2.074360,4.0,1.0
800,1 2000000,0,0.950234,0.431801,-2.530190,4.0,1.0


In [7]:
#Discretize x,y
NUMBER_OF_SQUARES = 100 #This should be a square number to create equal sized squares.
RESOLUTION_OF_S1 = 0.1 #This is used to discretize the yaw angle over 0 - 2*pi


df_discretized = df_clean.copy()
mappingBoxConstant = (NUMBER_OF_SQUARES ** (1/2.0)) / (LENGTH_OF_BOX)
for index, row in df_clean.iterrows():
    df_discretized.at[index, 'X'] = int(row['X'] * mappingBoxConstant)
    df_discretized.at[index, 'Y'] = int(row['Y'] * mappingBoxConstant)

df_discretized.head()

,Time,ID,X,Y,Yaw,ResetID,checkCorrectness
0,0 202000000,0,5.0,5.0,0.000000,4.0,1.0
200,0 402000000,0,6.0,4.0,-0.801413,4.0,1.0
400,0 602000000,0,7.0,3.0,-1.436840,4.0,1.0
600,0 802000000,0,7.0,3.0,-2.074360,4.0,1.0
800,1 2000000,0,8.0,3.0,-2.530190,4.0,1.0


In [8]:
#Create goal columns (x_t+1, y_t+1, z_t+1)

df_discretized['Mount_X_Next'] = df_discretized['X']
df_discretized['Mount_Y_Next'] = df_discretized['Y']
df_discretized['Mount_Yaw_Next'] = df_discretized['Yaw']

df_discretized['Mount_X_Next'].shift(-1)
df_discretized['Mount_Y_Next'].shift(-1)
df_discretized['Mount_Yaw_Next'].shift(-1)

df_discretized.drop(df_discretized.index[len(df_discretized)-1])
df_discretized.head()

,Time,ID,X,Y,Yaw,ResetID,checkCorrectness,Mount_X_Next,Mount_Y_Next,Mount_Yaw_Next
0,0 202000000,0,5.0,5.0,0.000000,4.0,1.0,5.0,5.0,0.000000
200,0 402000000,0,6.0,4.0,-0.801413,4.0,1.0,6.0,4.0,-0.801413
400,0 602000000,0,7.0,3.0,-1.436840,4.0,1.0,7.0,3.0,-1.436840
600,0 802000000,0,7.0,3.0,-2.074360,4.0,1.0,7.0,3.0,-2.074360
800,1 2000000,0,8.0,3.0,-2.530190,4.0,1.0,8.0,3.0,-2.530190


In [9]:
#one-hot encode the ID

In [11]:
#Split into training and testing
df_discretized['is_train'] = np.random.uniform(0, 1, len(df_discretized)) <= .75
df_discretized.head()

,Time,ID,X,Y,Yaw,ResetID,checkCorrectness,Mount_X_Next,Mount_Y_Next,Mount_Yaw_Next,is_train
0,0 202000000,0,5.0,5.0,0.000000,4.0,1.0,5.0,5.0,0.000000,True
200,0 402000000,0,6.0,4.0,-0.801413,4.0,1.0,6.0,4.0,-0.801413,True
400,0 602000000,0,7.0,3.0,-1.436840,4.0,1.0,7.0,3.0,-1.436840,True
600,0 802000000,0,7.0,3.0,-2.074360,4.0,1.0,7.0,3.0,-2.074360,True
800,1 2000000,0,8.0,3.0,-2.530190,4.0,1.0,8.0,3.0,-2.530190,True


In [12]:
#Create dataframes based on split
train, test = df_discretized[df_discretized['is_train']==True], df_discretized[df_discretized['is_train']==False]
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

('Number of observations in the training data:', 230716)
('Number of observations in the test data:', 76718)


In [13]:
#print list of information of each column (sanity check)
df_discretized.describe()

,ID,X,Y,Yaw,ResetID,checkCorrectness,Mount_X_Next,Mount_Y_Next,Mount_Yaw_Next
count,307434.0,307434.000000,307434.000000,307434.000000,307434.000000,307434.0,307434.000000,307434.000000,307434.000000
mean,0.0,5.595962,6.631700,-0.097234,52.874402,1.0,5.595962,6.631700,-0.097234
std,0.0,1.674328,1.651212,1.846034,41.798299,0.0,1.674328,1.651212,1.846034
min,0.0,1.000000,1.000000,-3.141580,4.000000,1.0,1.000000,1.000000,-3.141580
25%,0.0,4.000000,5.000000,-1.803095,24.000000,1.0,4.000000,5.000000,-1.803095
50%,0.0,5.000000,7.000000,-0.066676,43.000000,1.0,5.000000,7.000000,-0.066676
75%,0.0,8.000000,8.000000,1.512370,64.000000,1.0,8.000000,8.000000,1.512370
max,0.0,9.000000,9.000000,3.141580,204.000000,1.0,9.000000,9.000000,3.141580


In [14]:
# Create a list of the feature column's names
features = df_discretized.columns[2:5]

# View features
features

Index([u'X', u'Y', u'Yaw'], dtype='object')

In [15]:
#Train the classifier
clf = RandomForestClassifier(n_jobs=-1, verbose = 1)
clf.fit(train[features], train['Mount_X_Next'])

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [16]:
# Apply the Classifier we trained to the test data 
preds= clf.predict(test[features])

[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.1s finished


In [17]:
# View the predicted probabilities of the first 10 observations
clf.predict_proba(test[features])[0:10]

[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.1s finished


array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [18]:
# Create confusion matrix
pd.crosstab(test['Mount_X_Next'], preds, rownames=['Actual Species'], colnames=['Predicted Species'])

Predicted Species,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
Actual Species,,,,,,,,,
1.0,589,0,0,0,0,0,0,0,0
2.0,0,309,0,0,0,0,0,0,0
3.0,0,0,864,0,0,0,0,0,0
4.0,0,0,0,22556,0,0,0,0,0
5.0,0,0,0,0,24607,0,0,0,0
6.0,0,0,0,0,1,1293,0,0,0
7.0,0,0,0,0,0,4,7018,0,0
8.0,0,0,0,0,0,0,0,19470,0
9.0,0,0,0,0,0,0,0,0,7


In [19]:
# View a list of the features and their importance scores
list(zip(train[features], clf.feature_importances_))

[('X', 0.8687527764537268),
 ('Y', 0.0661201934669394),
 ('Yaw', 0.06512703007933371)]